In [1]:
import nltk.data
import re
import pandas as pd
from nltk.tokenize import sent_tokenize
import numpy as np
import torch
import json
from sentence_transformers import SentenceTransformer
import tensorflow as tf

In [26]:
def load_jsonl(data_path):
    data = []
    with open(data_path) as f:
        for line in f:
            data.append(json.loads(line))
    return data
model = SentenceTransformer('all-MiniLM-L6-v2')
loss_func = torch.nn.MarginRankingLoss(0.0)
cos = torch.nn.CosineSimilarity(dim=0)

In [176]:
data = load_jsonl("reddit_data/reddit_rouge_100.jsonl")
loss = []
doc_embed = []
summary_embed = []
for example in data:
    gold_loss = []
    candidates = []
    doc = example['text']
    doc_tokens = sent_tokenize(doc)
    sentence_embeddings = torch.tensor(model.encode(doc_tokens))
    doc_embedding = torch.mean(sentence_embeddings,0)
    doc_embed.append(doc_embedding)
    
    gold_summary = example['summary']
    gold_tokens = sent_tokenize(gold_summary)
    gold_sent_embeddings = torch.tensor(model.encode(gold_tokens))
    gold_embedding = torch.mean(gold_sent_embeddings,0)
    
    cos2 = cos(doc_embedding, gold_embedding)
    for summaries in example['scores']:
        summary_tokens = sent_tokenize(summaries[0])
        summary_sent_embeddings = torch.tensor(model.encode(summary_tokens))
        summary_embedding = torch.mean(summary_sent_embeddings,0)
        candidates.append(summary_embedding)
        
        cos1 = cos(doc_embedding, summary_embedding)
        gold_loss.append(abs(cos2-cos1))

    loss.append(gold_loss)
    summary_embed.append(candidates)

In [190]:
loss
index_sum = []
for losses in loss:
    min_loss = min(losses,default=-1)
    if (min_loss != -1):
        index_sum.append(losses.index(min_loss))
    else:
        index_sum.append(min_loss)

matchsum = []
for i in range(100):
    pair = {}
    pair['text'] = data[i]['text']
    if index_sum[i] == -1:
        pair['summary'] = data[i]['idx']
    else:
        pair['summary'] = data[i]['scores'][index_sum[i]][0]
    matchsum.append(pair)

matchsum

[{'text': 'this actually happened a couple of years ago. i grew up in germany where i went to a german secondary school that went from 5th to 13th grade (we still had 13 grades then, they have since changed that). my school was named after anne frank and we had a club that i was very active in from 9th grade on, which was dedicated to teaching incoming 5th graders about anne franks life, discrimination, anti-semitism, hitler, the third reich and that whole spiel. basically a day where the students\' classes are cancelled and instead we give them an interactive history and social studies class with lots of activities and games. \n\nthis was my last year at school and i already had a lot of experience doing these project days with the kids. i was running the thing with a friend, so it was just the two of us and 30-something 5th graders. we start off with a brief introduction and brainstorming: what do they know about anne frank and the third reich? you\'d be surprised how much they know.

In [192]:
df_matchsum = pd.DataFrame(matchsum, columns=['text','summary'])
df_matchsum.to_json("reddit_data/matchsum_100.jsonl", orient='records', lines=True)

## 